[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/sensioai/blog/blob/master/043_cnn_arquitecturas/arquitecturas.ipynb)

# Arquitecturas de Redes Convolucionales

En el post [anterior](https://sensioai.com/blog/042_cnns) hemos introducido la arquitectura de `red neuronal convolucional`, un modelo de `red neuronal` especialmente diseñado para trabajar con imágenes en tareas de visión artificial. Entre estas tareas, la más común es la de clasificación de imágenes, que consiste simplemente en asignar una categoría de entre varias a una imagen en particular. Una buena manera para medir el progreso en el campo del `Machine Learning` es a través de `benchmarks` y competiciones en los que investigadores proponen diferentes soluciones a un mismo problema, que se evalúan sobre un mismo conjunto de datos de test oculto. En el campo del `deep learning` y `computer vision`, una de estas competiciones, que hoy en día ya es un `benchmark` establecido, es la clasificación de imágenes en el dataset [Imagenet](http://www.image-net.org/challenges/LSVRC/). En 2012, la primera `red neuronal convolucional` entró en esta competición consiguiendo una mejora de 10 puntos sobre la mejor solución hasta la fecha, lo cual supuso un detonante para el campo del `deep learning`. A partir de ese momento, todas las soluciones ganadoras presentadas durante los años siguientes han sido redes convolucionales, convirtiendo a Imagenet una estupenda fuente de información para conocer los últimos avances en el campo (cuando una nueva arquitectura es diseñada, es práctica común reportar sus prestaciones en Imagenet para poder compararla con otras redes). En este post vamos a revisar algunas de las arquitecturas más conocidas que han aparecido durante los años, las cuales podremos usar para nuestras aplicaciones. Puedes conocer en detalle del progreso de este `benchmark` en [papers with code](https://paperswithcode.com/sota/image-classification-on-imagenet), donde también encontrarás artículos y código en muchísimos otros campos de aplicación.

![](https://www.researchgate.net/profile/Kien_Nguyen26/publication/321896881/figure/fig1/AS:573085821489153@1513645715549/The-evolution-of-the-winning-entries-on-the-ImageNet-Large-Scale-Visual-Recognition.png)

## LeNet-5

Empezamos revisado la primera arquitectura de `CNN` desarrollada. Esta red es conocida por el nombre de LeNet-5 y, si bien no participó en la competición de Imagenet, es interesante conocer esta primera arquitectura.

![](https://pythonmachinelearning.pro/wp-content/uploads/2017/09/lenet-5.png.webp)

Esta `CNN` es alimentada por imágenes de 32x32 píxeles de dígitos manuscritos, y está formada por tres capas convolucionales, dos de las cuales reducen la dimensionalidad de los mapas de características mediante *average pooling*. La salida de la última capa convolucional es conectada un perceptrón multicapa de dos capas, que tienen la responsabilidad de dar la predicción final. Puedes ver más detalles en el [artículo](http://yann.lecun.com/exdb/publis/pdf/lecun-01a.pdf) original. A continuación puedes ver un ejemplo de implementación en `Pytorch`.

In [ ]:
import torch 

def block(c_in, c_out, k=3, p=1, s=1):
    return torch.nn.Sequential(
        torch.nn.Conv2d(c_in, c_out, k, padding=p, stride=s),
        torch.nn.Tanh(),
        torch.nn.AvgPool2d(2, stride=2)
    )

def block2(c_in, c_out):
    return torch.nn.Sequential(
        torch.nn.Linear(c_in, c_out),
        torch.nn.ReLU()
    )

class LeNet5(torch.nn.Module):
  def __init__(self, n_channels=1, n_outputs=10):
    super().__init__()
    #self.pad = torch.nn.ConstantPad2d(2, 0.)
    self.conv1 = block(n_channels, 6, 5, p=0)
    self.conv2 = block(6, 16, 5, p=0)
    self.conv3 = torch.nn.Sequential(
        torch.nn.Conv2d(16, 120, 5, padding=0),
        torch.nn.Tanh()
    )
    self.fc1 = block2(120, 84)
    self.fc2 = torch.nn.Linear(84, 10)

  def forward(self, x):
    #x = self.pad(x)
    x = self.conv1(x)
    x = self.conv2(x)
    x = self.conv3(x)
    x = x.view(x.shape[0], -1)
    x = self.fc1(x)
    x = self.fc2(x)
    return x

In [ ]:
lenet5 = LeNet5()
output = lenet5(torch.randn(64, 1, 32, 32))
output.shape

torch.Size([64, 10])

## AlexNet

[AlexNet](https://papers.nips.cc/paper/4824-imagenet-classification-with-deep-convolutional-neural-networks.pdf) ganó la competición de Imagenet en 2012. Es una arquitectura similar a LeNet5 pero más profunda. Fue la primera red convolucional que ganó la competición, y también la primera en ser entrenada en GPUs.

![](https://miro.medium.com/fit/c/1838/551/1*arJJYgK-_7VcuKKX1TCKMA.png)

Tanto esta red como muchas otras las podemos encontrar ya implementadas en varios paquetes. En `Pytorch` podemos descargar redes convolucionales con el paquete `torchvision`.

In [ ]:
import torchvision

alexnet = torchvision.models.AlexNet()
alexnet

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [ ]:
output = alexnet(torch.randn(64, 3, 224, 224))
output.shape

torch.Size([64, 1000])

> 💡El dataset Imagenet contiene imágenes comunes descargadas de internet, y el objetivo es el de clasificar imágenes entre 1000 clases diferentes. Es por este motivo que verás que los diferentes modelos en `torchvision` tienen una última capa lineal con 1000 salidas.

## GoogLeNet

Ganadora de la edición de 2014, GoogLeNet es una red mucho más profunda (podemos empezar a ver la tendencia de que cuantas más capas, mejor resultados). Sus desarrolladores implementaron los módulos *inception*, los cuales aplican diferentes capas convolucionales, con diferentes tamaños de filtros, en paralelo sobre las mismas entradas y luego concatenan las salidas. Además, para poder entrenar esta red tan profunda, se llevan a cabo predicciones en diversas capas intermedias para poder tener gradientes fluyendo hacia las capas iniciales. 

![](https://www.alanzucconi.com/wp-content/uploads/2015/07/googlenet-arch.png)

In [ ]:
googlenet = torchvision.models.GoogLeNet()
googlenet

GoogLeNet(
  (conv1): BasicConv2d(
    (conv): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (conv2): BasicConv2d(
    (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (conv3): BasicConv2d(
    (conv): Conv2d(64, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(192, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (inception3a): Inception(
    (branch1): BasicConv2d(
      (conv): Conv2d(192, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track

In [ ]:
output = googlenet(torch.randn(64, 3, 224, 224))
output[0].shape

torch.Size([64, 1000])

## VGG

Si bien [VGG](https://arxiv.org/abs/1409.1556) no ganó, tuvo mucho éxito ya que sentó un precedente en el patrón del diseño de las redes convolucionales. Este patrón consiste en usar siempre capas convolucionales con filtros de 3x3, *stride* y *padding* 1 (manteniendo el tamaño de las entradas constante) y usar *max pool* para reducir a la mitad los mapas de características. Además, después de cada capa se dobla el número de filtros.

![](http://jesusutrera.com/articles/img/vgg_model.png)

In [ ]:
# existen dos variantes: vgg16 y vgg19, con 16 y 19 capas respectivamente

vgg16 = torchvision.models.vgg16()
vgg16

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [ ]:
output = vgg16(torch.randn(64, 3, 224, 224))
output.shape

torch.Size([64, 1000])

## ResNet

[ResNet](https://arxiv.org/abs/1512.03385) fue el campeón en 2015. La variante ganadora tenía 152 capas y confirmó la tendencia de profundizar con menos parámetros. La clave para entrenar redes tan profundas es el uso de *skip connections* donde la señal que entra a una capa también se agrega a la salida. Esto proporciona una ruta limpia para que el gradiente se propague desde la salida a la entrada. También se puede interpretar como una forma de dejar a la red la decisión de qué capas usar (si alguna capa no es necesaria, simplemente se la puede saltar).

![](https://i.stack.imgur.com/XTo6Q.png)

In [ ]:
# existen varias variantes: resnet18, resnet35, resnet50, resnet101, resnet152

resnet34 = torchvision.models.resnet34()
resnet34

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [ ]:
output = resnet34(torch.randn(64, 3, 224, 224))
output.shape

torch.Size([64, 1000])

## Otras arquitecturas

Tras la finalización de la competición de Imagenet, nuevas arquitecturas han aparecido (y siguen apareciendo) que mejoran considerablemente las redes vistas anteriormente. Algunos ejemplos son:

- Xception (2016): mejora sobre GoogLeNet (básicamente, GoogLeNet + ResNet)
- SENet (2017): Añade los bloques *SE* a GoogLeNet y ResNet.
- MobileNet (2017): família de CNNs especialmente diseñadas para trabajar en dispositivos con pocos recursos computacionales como smartphones. 
- EfficientNet (2019): família de redes diseñadas mediante algoritmos de `Inteligencia Artificial`(IA haciendo IA 🤯). Mejores prestaciones con menos parámetros.

## Resumen

En este post hemos presentado diferentes arquitecturas de `redes convolucionales` que se han desarrollado durante los últimos años. Estas redes han destacado por sus buenos resultados en el dataset Imagenet, una competición muy conocida en el mundo del `deep learning` para `computer vision` y `benchmark` establecido en el que todas las nuevas arquitecturas se evalúan para poder compararlas. Como hemos visto, las redes con mejores resultados son redes muy profundas (con muchas capas) y parámetros. Esto es un problema si queremos entrenar una de ellas desde cero en algún problema concreto. Afortunadamente, no sólo podemos aprovechar la arquitectura sino que podemos descargar también los parámetros entrenados y utilizarlos como estado inicial para nuestra aplicación. Esta técnica es conocida como transfer learning, y nos permite conseguir buenos modelos con menor requisitos computacionales. En el próximo post veremos como llevar a cabo esta técnica de manera sencilla.